In [1]:
import tensorflow as tf
import numpy as np
import os
from tensorflow.python.framework.convert_to_constants import convert_variables_to_constants_v2
from tensorflow.keras import layers, models, regularizers, optimizers
from sklearn.model_selection import train_test_split
import h5py
import H5pyHelper
from datetime import datetime
import tf2onnx
from  tensorrt import OnnxParser
from sklearn.metrics import mean_squared_error, r2_score

print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
BATCH_SIZE = 32

Num GPUs Available:  1


In [2]:
# A memory-mapped array is kept on disk. However, it can be accessed and sliced like any ndarray.
# Memory mapping is especially useful for accessing small fragments of large files without reading
# the entire file into memory.

def generator(feature_name,label_name,batch_size,shuffle):
    index = 0
    while True:
        
        if index == 0:
            if (shuffle):
                ## shuffle each epoch
                seed = datetime.now()
                
                H5pyHelper.shuffle_dataset(
                    "/home/sorozco0612/dev/alan_bot/data_processing/road_data/data.h5", "x_train", seed
                )
                H5pyHelper.shuffle_dataset(
                    "/home/sorozco0612/dev/alan_bot/data_processing/road_data/data.h5", "y_train", seed
                )
            hf = h5py.File('/home/sorozco0612/dev/alan_bot/data_processing/road_data/data.h5', 'r')
    
            x = hf[feature_name]
            y = hf[label_name]      
        
        # batch has not met the end of the data
        if (index + batch_size < x.shape[0]):
            features = x[index:index+batch_size]
            labels = y[index:index+batch_size]
            index += batch_size
        else:
            # batch size will be smaller than the rest on last iteration
            features = x[index:]
            labels = y[index:]
            index = 0
            
            ## close file so it can be reshuffled
            hf.close()
            
            

        
        yield (features,labels)
        
def generator(feature_name,label_name,batch_size):
    index = 0
    while True:
        if index == 0:
            print("1")
            hf = h5py.File('/home/sorozco0612/dev/alan_bot/data_processing/road_data/data.h5', 'r')
    
            x = hf[feature_name]
            y = hf[label_name]
            
            
            idx_map = np.arange(x.shape[0])

            np.random.shuffle(idx_map)
            print("2")
            
        
        # batch has not met the end of the data
        if (index + batch_size < x.shape[0]):
            print("3")
            features = []
            labels = []
            
            idx_map[index:index + batch_size].sort()
            batch = idx_map[index:index + batch_size]
            
            features = x[batch]
            labels = y[batch]
            
            #print(x[0].shape)
            #for i in idx_map[index:index + batch_size]:
            #    print(i)
            #    features.append(x[i])
            #    labels.append(y[i])
                
            #labels = [y[i] for i in idx_map[index:index+batch_size]]
            index += batch_size
            print("4")
            
        else:
            # batch size will be smaller than the rest on last iteration
            features = [x[i] for i in idx_map[index:]]
            labels = [y[i] for i in idx_map[index:]]
            index = 0
            
            ## close file so it can be reshuffled
            hf.close()
            
        print(np.array(features).shape)
        print(np.array(labels).shape)

        yield (np.array(features),np.array(labels))
        

train_generator = generator('x_train', 'y_train',BATCH_SIZE)
test_generator = generator('x_test','y_test',BATCH_SIZE)


In [3]:
model = models.Sequential()

#model.add(layers.Conv2D(32,(3,3),activation='relu', input_shape=(432,614,3), kernel_regularizer=regularizers.l2(0.00020)))
#model.add(layers.MaxPool2D((2,2)))
#model.add(layers.Dropout(0.20))
#          
#model.add(layers.Conv2D(64,(3,3),activation='relu', kernel_regularizer=regularizers.l2(0.00020)))
#model.add(layers.MaxPool2D((2,2)))
#model.add(layers.Dropout(0.30))
#          
#model.add(layers.Conv2D(128, kernel_size=(3, 3), activation='relu' ,kernel_regularizer=regularizers.l2(0.00020)))
#model.add(layers.MaxPool2D((2,2)))
#model.add(layers.Dropout(0.40))
#          
#model.add(layers.Flatten())
#model.add(layers.Dense(128,activation='linear' ,kernel_regularizer=regularizers.l2(0.00020)))
#model.add(layers.Dropout(0.30))
#model.add(layers.Dense(64,activation='linear' ,kernel_regularizer=regularizers.l2(0.00020)))
#model.add(layers.Dropout(0.30))

# Convolution Layers
model.add(layers.Conv2D(24, (5, 5), strides=(2, 2), input_shape=(66,200, 3), activation='elu')) 
model.add(layers.Conv2D(36, (5, 5), strides=(2, 2), activation='elu')) 
model.add(layers.Conv2D(48, (5, 5), strides=(2, 2), activation='elu')) 
model.add(layers.Conv2D(64, (3, 3), activation='elu')) 
model.add(layers.Dropout(0.2)) # not in original model. added for more robustness
model.add(layers.Conv2D(64, (3, 3), activation='elu')) 
    
# Fully Connected Layers
model.add(layers.Flatten())
model.add(layers.Dropout(0.2)) # not in original model. added for more robustness
model.add(layers.Dense(100, activation='elu'))
model.add(layers.Dense(50, activation='elu'))
model.add(layers.Dense(10, activation='elu'))

model.add(layers.Dense(1))

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 31, 98, 24)        1824      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 14, 47, 36)        21636     
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 5, 22, 48)         43248     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 3, 20, 64)         27712     
_________________________________________________________________
dropout (Dropout)            (None, 3, 20, 64)         0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 1, 18, 64)         36928     
_________________________________________________________________
flatten (Flatten)            (None, 1152)              0

In [4]:
optimizer = optimizers.Adam(lr=1e-4) # lr is learning rate
model.compile(optimizer=optimizer,
              loss = tf.keras.losses.MeanSquaredError(),
              metrics=['mean_squared_error'])

In [5]:
# Include the epoch in the file name (uses `str.format`)
#checkpoint_path = "training/cp-{epoch:04d}.ckpt"
#checkpoint_dir = os.path.dirname(checkpoint_path)

# Create a callback that saves the model's weights every 5 epochs
#cp_callback = tf.keras.callbacks.ModelCheckpoint(
 #   filepath=checkpoint_path, 
 #   verbose=1, 
 #   save_weights_only=True,
 #   save_freq=5*batch_size)

# Save the weights using the `checkpoint_path` format
#model.save_weights(checkpoint_path.format(epoch=0))

hf = h5py.File('/home/sorozco0612/dev/alan_bot/data_processing/road_data/data.h5', 'r')
train_size = hf["x_train"].shape[0]
test_size = hf["x_test"].shape[0]
hf.close()

callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)

# Train model with new callback
#fit = model.fit(x_train,
 #               y_train,
  #              epochs=50, 
  #              #callbacks=[cp_callback], 
  #              verbose=2,
  #              validation_data=(x_test,y_test))
fit = model.fit(x=train_generator,
                steps_per_epoch= train_size // BATCH_SIZE,
                callbacks=[callback], 
                validation_steps= test_size // BATCH_SIZE,
                validation_data=test_generator,
                verbose=1,
                epochs=250)

1
2
3
4
(32, 66, 200, 3)
(32,)
Epoch 1/250
3
4
(32, 66, 200, 3)
(32,)


UnknownError:  Failed to get convolution algorithm. This is probably because cuDNN failed to initialize, so try looking to see if a warning log message was printed above.
	 [[node sequential/conv2d/Conv2D (defined at <ipython-input-5-526586d81016>:35) ]] [Op:__inference_train_function_1182]

Function call stack:
train_function


In [ ]:
def summarize_prediction(Y_true, Y_pred):
    mse = mean_squared_error(Y_true, Y_pred)
    r_squared = r2_score(Y_true, Y_pred)
    print(f'mse       = {mse:.2}')
    print(f'r_squared = {r_squared:.2%}')
    
def predict_and_summarize(X, Y):
    Y_pred = model.predict(X)
    summarize_prediction(Y, Y_pred)
    return Y_pred

hf = h5py.File('/home/sorozco0612/dev/alan_bot/data_processing/road_data/data.h5', 'r')
x_test = hf["x_test"]
y_test = hf["y_test"]

y_pred = predict_and_summarize(x_test, y_test)

print(y_pred[:10])
print(y_test[:10])

hf.close()

In [ ]:
## save the model regularly
model.save('/home/sorozco0612/dev/alan_bot/model_creation/alan_bot_model')

In [ ]:
## convert saved model to ONNX
!python3 -m tf2onnx.convert --saved-model "/home/sorozco0612/dev/alan_bot/model_creation/alan_bot_model" --output "alan_bot_model_onnx.onnx"